In [1]:
import os
import re
import torch
import transformers
from tqdm import tqdm
import json
import numpy as np
import polars as pl
from sklearn.metrics import cohen_kappa_score
from typing import Optional, Literal
from utils import load_asap_dataset, load_toefl_dataset, get_score_range
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

In [8]:
df = load_asap_dataset('datasets/ASAP', stratify=True)

Could not determine dtype for column 5, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 11, falling back to string
Could not determine dtype for column 12, falling back to string
Could not determine dtype for column 13, falling back to string
Could not determine dtype for column 14, falling back to string
Could not determine dtype for column 15, falling back to string
Could not determine dtype for column 16, falling back to string
Could not determine dtype for column 17, falling back to string
Could not determine dtype for column 18, falling back to string
Could not determine dtype for column 19, falling back to string
Could not determine dtype for column 20, falling back to string
Could not determine dtype for column 21, falling back to string
Could not determine dtype for column 22, falling back to string
Could not determine dtype for column 23, falling back to string
Could not determine dtype for column 24, 

In [9]:
df

essay_set,essay_id,essay,score
i64,i64,str,i64
8,20826,""" Bell rings. Shuffle, shuffle…",60
4,10064,"""The author concludes the story…",1
3,6127,"""The features of the setting in…",3
5,13551,"""The mood created by the author…",1
6,16370,"""some of the obstacles the buil…",2
…,…,…,…
2,3090,"""They were talking about thinki…",3
3,6187,"""The features of the setting af…",1
7,18150,"""One @DATE1 @TIME1 I was very p…",14


In [44]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-3.1-8B-Instruct"
# model_name = "meta-llama/Llama-3.2-3B-Instruct"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
results = pl.read_csv(f"outputs/vanilla/{model_name.split('/')[1]}.csv")
results = results.rename({"score": "pred_score"})
results

essay_set,essay_id,response,pred_score
i64,i64,str,i64
8,20826,"""assistant Evaluation: The es…",1
4,10064,"""assistant Evaluation: The st…",1
3,6127,"""assistant Evaluation: The st…",2
5,13551,"""assistant Evaluation: The st…",1
6,16370,"""assistant Evaluation: The st…",60
…,…,…,…
2,3090,"""assistant Evaluation: The st…",1
3,6187,"""assistant Evaluation: The st…",1
7,18150,"""assistant Evaluation: The es…",16


In [45]:
results.describe()

statistic,essay_set,essay_id,response,pred_score
str,f64,f64,str,f64
"""count""",1299.0,1299.0,"""1299""",1299.0
"""null_count""",0.0,0.0,"""0""",0.0
"""mean""",4.163202,10237.457275,null,15.108545
"""std""",2.143791,6344.699906,null,145.010105
"""min""",1.0,1.0,"""assistant Evaluation: The es…",0.0
"""25%""",2.0,4404.0,null,1.0
"""50%""",4.0,9934.0,null,1.0
"""75%""",6.0,15784.0,null,3.0
"""max""",8.0,21599.0,"""assistant Evaluation: The stu…",1961.0


In [46]:
for_evaluation = df.join(results, on=['essay_set', 'essay_id'])
for_evaluation

essay_set,essay_id,essay,score,response,pred_score
i64,i64,str,i64,str,i64
8,20826,""" Bell rings. Shuffle, shuffle…",60,"""assistant Evaluation: The es…",1
4,10064,"""The author concludes the story…",1,"""assistant Evaluation: The st…",1
3,6127,"""The features of the setting in…",3,"""assistant Evaluation: The st…",2
5,13551,"""The mood created by the author…",1,"""assistant Evaluation: The st…",1
6,16370,"""some of the obstacles the buil…",2,"""assistant Evaluation: The st…",60
…,…,…,…,…,…
2,3090,"""They were talking about thinki…",3,"""assistant Evaluation: The st…",1
3,6187,"""The features of the setting af…",1,"""assistant Evaluation: The st…",1
7,18150,"""One @DATE1 @TIME1 I was very p…",14,"""assistant Evaluation: The es…",16


In [47]:
for_evaluation = for_evaluation.filter(pl.col("pred_score") != -1)
for_evaluation

essay_set,essay_id,essay,score,response,pred_score
i64,i64,str,i64,str,i64
8,20826,""" Bell rings. Shuffle, shuffle…",60,"""assistant Evaluation: The es…",1
4,10064,"""The author concludes the story…",1,"""assistant Evaluation: The st…",1
3,6127,"""The features of the setting in…",3,"""assistant Evaluation: The st…",2
5,13551,"""The mood created by the author…",1,"""assistant Evaluation: The st…",1
6,16370,"""some of the obstacles the buil…",2,"""assistant Evaluation: The st…",60
…,…,…,…,…,…
2,3090,"""They were talking about thinki…",3,"""assistant Evaluation: The st…",1
3,6187,"""The features of the setting af…",1,"""assistant Evaluation: The st…",1
7,18150,"""One @DATE1 @TIME1 I was very p…",14,"""assistant Evaluation: The es…",16


In [48]:
from scipy.stats import spearmanr
# essay_setごとにQWKとスピアマンの順位相関係数を計算
qwk_scores = []
spearman_scores = []
for essay_set in for_evaluation['essay_set'].unique():
    subset = for_evaluation.filter(pl.col('essay_set') == essay_set)
    qwk = cohen_kappa_score(
        subset['score'].to_numpy(),
        subset['pred_score'].to_numpy(),
        weights='quadratic',
        labels=np.arange(get_score_range("ASAP", essay_set)[0], get_score_range("ASAP", essay_set)[1] + 1)
    )
    spearman_corr, _ = spearmanr(subset['score'].to_numpy(), subset['pred_score'].to_numpy())
    qwk_scores.append({
        'essay_set': essay_set,
        'qwk': qwk
    })
    spearman_scores.append({
        'essay_set': essay_set,
        'spearman_corr': spearman_corr
    })

qwk_df = pl.DataFrame(qwk_scores)
spearman_df = pl.DataFrame(spearman_scores)
print("QWK scores by essay set:")
print(qwk_df)
print("スピアマンの順位相関係数 by essay set:")
print(spearman_df)

QWK scores by essay set:
shape: (8, 2)
┌───────────┬───────────┐
│ essay_set ┆ qwk       │
│ ---       ┆ ---       │
│ i64       ┆ f64       │
╞═══════════╪═══════════╡
│ 1         ┆ 0.129049  │
│ 2         ┆ 0.023058  │
│ 3         ┆ 0.242967  │
│ 4         ┆ 0.550107  │
│ 5         ┆ 0.301015  │
│ 6         ┆ 0.341296  │
│ 7         ┆ 0.006005  │
│ 8         ┆ -0.041585 │
└───────────┴───────────┘
スピアマンの順位相関係数 by essay set:
shape: (8, 2)
┌───────────┬───────────────┐
│ essay_set ┆ spearman_corr │
│ ---       ┆ ---           │
│ i64       ┆ f64           │
╞═══════════╪═══════════════╡
│ 1         ┆ 0.004657      │
│ 2         ┆ 0.050311      │
│ 3         ┆ 0.450554      │
│ 4         ┆ 0.618181      │
│ 5         ┆ 0.423984      │
│ 6         ┆ 0.428682      │
│ 7         ┆ 0.061414      │
│ 8         ┆ -0.090297     │
└───────────┴───────────────┘


In [49]:
print(f"QWK mean: {qwk_df['qwk'].mean():.3f}")
print(f"スピアマンの順位相関係数 mean: {spearman_df['spearman_corr'].mean():.3f}")

QWK mean: 0.194
スピアマンの順位相関係数 mean: 0.243
